# Adding Locations Columns

The goal is to predict SalePrice

In [2]:
KAGGLE=False # set to True if using the Kaggle version of the dataset

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import sklearn.preprocessing

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, StratifiedKFold,KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


In [4]:
import catboost as cb

from xgboost import XGBRegressor


In [5]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)

In [133]:
def load_data(tree=None):
    if KAGGLE:
        train = pd.read_csv('data/train.csv')
        test = pd.read_csv('data/test.csv')
        num_train=np.shape(train)[0]
        num_test=np.shape(test)[0]
        alldata=train.append(test, ignore_index=True)
    else:
        alldata=pd.read_csv('./data/Ames_HousePrice.csv', index_col=0)
        #locations=pd.read_csv('./data/Ames_Real_Estate_Data.csv', index_col=0)
    return alldata

alldata=load_data()

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [91]:
locations=pd.read_csv('./data/Ames_Real_Estate_Data.csv', index_col=0)


/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [134]:
locations['Prop_Addr']=locations['Prop_Addr'].astype(str)
locations['MA_City']=locations['MA_City'].astype(str)
locations['MA_State']=locations['MA_State'].astype(str)
locations['MA_Zip1']=locations['MA_Zip1'].astype(str).map(lambda x:x[:5])

In [93]:
locations.columns



Index(['GeoRefNo', 'Tier', 'Range', 'Prop_Addr', 'ZngCdPr', 'ZngCdSc',
       'ZngOLPr', 'ZngOLSc', 'ClassPr_S', 'ClassSc_S', 'Legal_Pr', 'SchD_S',
       'TxD_S', 'MA_Ownr1', 'MA_Ownr2', 'MA_Line1', 'MA_Line2', 'MA_City',
       'MA_State', 'MA_Zip1', 'MA_Zip2', 'Rcrd_Yr', 'Rcrd_Mo', 'Inst1_No',
       'Inst1_Yr', 'Inst1_Mo', 'Inst1TPr', 'LndAc_S', 'ImpAc_S', 'OthAc_S',
       'TtlVal_AsrYr', 'ValType', 'X1TPr_D', 'X1TSc_D', 'X2TPr_D', 'X2TSc_D',
       'X1TPr_S', 'X1TSc_S', 'X2TPr_S', 'X2TSc_S', 'LndAcX1S', 'ImpAcX1S',
       'ImpAcX2S', 'HSTtl_D', 'MilVal_D', 'HSTtl_S', 'MilVal_S', 'AcreX_S1',
       'AcreGr', 'AcreNt_S', 'Neighborhood', 'LotArea', 'ParType', 'BldgNo_S',
       'DwlgNo_S', 'BldgType', 'YrBuilt', 'HouseStyle', 'Foundation',
       'RoofMatl', 'Ext1', 'Ext2', 'MasVnrType', 'Heating', 'Central Air',
       'GLA', 'TtlBsmtSF', 'TotRmsAbvGrd', 'Fireplaces', 'PoolArea',
       'GarageType', 'GarYrBlt', 'Cars', 'GarageArea', 'YrSold_YYYY',
       'MoSold_MM', 'SalePrice', 

In [138]:
alldata.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
2,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
3,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
4,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
5,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal


In [140]:
alldata=alldata.merge(locations2,left_on='PID',right_on='MapRefNo')

In [142]:
np.shape(alldata)

(2603, 83)

In [141]:
alldata.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Full_Addr,MA_Ownr1
0,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal,436 HAYWARD AVE AMES IA 50010,"GREINER, RODNEY & CELIA"
1,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal,3416 WEST ST AMES IA 50014,"HUBERT, JOHN F & GRETTA R"
2,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal,3416 WEST ST AMES IA 50014,"HUBERT, JOHN F & GRETTA R"
3,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal,320 S 2ND ST AMES IA 50010,"SKLENAR, TIM"
4,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal,1524 DOUGLAS AVE AMES IA 50010,"MILLARD, TAMMY"


In [131]:
locations2.head(30)

,Full_Addr,MA_Ownr1
MapRefNo,,
520400001,4599 GEORGE W CARVER AVE AMES IA 50010,HUNZIKER DEVELOPMENT GROUP LLC
520440010,4507 EVEREST AVE AMES IA 50010,HUNZIKER DEVELOPMENT GROUP LLC
520440020,4513 EVEREST AVE ANKENY IA 50023,HAPPE HOMES LLP
520445001,3449 BARCELOS ST AMES IA 50010,SCENIC VALLEY PROPERTY OWNERS ASSOCIATION INC
520445010,4506 EVEREST AVE ANKENY IA 50023,HAPPE HOMES LLP
520445020,4512 EVEREST AVE ANKENY IA 50023,HAPPE HOMES LLP
520450001,3448 BARCELOS ST AMES IA 50010,SCENIC VALLEY PROPERTY OWNERS ASSOCIATION INC
520450010,4421 EVEREST CIR ANKENY IA 50023,HAPPE HOMES LLP
520450020,4415 EVEREST CIR AMES IA 50010,HUNZIKER CONSTRUCTIONS SERVICES INC


In [ ]:
#lats
locations['loc_obj'].map(lambda x: x.latitude if x is not None)

In [159]:
alldata.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Full_Addr,MA_Ownr1,objs,coords,lats,longs
0,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal,436 HAYWARD AVE AMES IA 50010,"GREINER, RODNEY & CELIA","(436, Hayward Avenue, Union Drive, Ames, Story...","(42.01856448979592, -93.65161918367347)",42.018564,-93.651619
1,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal,3416 WEST ST AMES IA 50014,"HUBERT, JOHN F & GRETTA R","(3416, West Street, Union Drive, Ames, Story C...","(42.024855315930424, -93.66367099341755)",42.024855,-93.663671
2,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal,3416 WEST ST AMES IA 50014,"HUBERT, JOHN F & GRETTA R","(3416, West Street, Union Drive, Ames, Story C...","(42.024855315930424, -93.66367099341755)",42.024855,-93.663671
3,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal,320 S 2ND ST AMES IA 50010,"SKLENAR, TIM","(320, South 2nd Street, Ames, Story County, Io...","(42.021547615297024, -93.61406809992513)",42.021548,-93.614068
4,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal,1524 DOUGLAS AVE AMES IA 50010,"MILLARD, TAMMY","(1524, Douglas Avenue, Broadmoor, Ames, Story ...","(42.03739146938776, -93.61220695918368)",42.037391,-93.612207


In [171]:
#Approach inspired by https://stackoverflow.com/questions/30218394/timeout-error-in-python-geopy-geocoder
#Repeated object creation has a side effect of slowing down the code enough to not exceed the Nominatim's quota
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

def do_geocode(address):
    geopy = Nominatim(user_agent="ames_housing_prices",timeout = 10)
    try:
        obj=geopy.geocode(address)
        #print(obj,end=" ")
        return geopy.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)
    
#alldata['objs'] = alldata['Full_Addr'].apply(do_geocode)

In [155]:
alldata['coords'] = alldata['objs'].apply(lambda x: (x.latitude, x.longitude) if x is not None else None)

alldata['lats'] = alldata['coords'].apply(lambda x: x[0] if x is not None else None)
alldata['longs'] = alldata['coords'].apply(lambda x: x[1] if x is not None else None)

#alldata.to_csv("./data/Ames_with_locations",index=False)

alldata2=alldata.drop(['objs','coords'],axis=1)

alldata2=alldata2.drop(['MA_Ownr1','Full_Addr'],axis=1)

In [204]:
alldata.to_csv("./data/Ames_HousePriceLocsFULL.csv",index=False)

In [172]:
alldata2.to_csv("./data/Ames_HousePriceLocs.csv",index=False)

In [173]:
alldata2.lats.isnull().sum()

256

In [182]:
mean_lats=alldata2.lats.mean()

In [181]:
mean_longs=alldata2.longs.mean()

In [191]:
mean_longs

-93.65146413611599

In [197]:
alldata2['MissingLoc']=np.where(alldata.coords.isnull(), 1, 0)


In [196]:
alldata.head()

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Full_Addr,MA_Ownr1,objs,coords,lats,longs
0,856,126000,30,RL,65.556308,7890,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,None,None,None,0,3,2010,WD,Normal,436 HAYWARD AVE AMES IA 50010,"GREINER, RODNEY & CELIA","(436, Hayward Avenue, Union Drive, Ames, Story...","(42.01856448979592, -93.65161918367347)",42.018564,-93.651619
1,1049,139500,120,RL,42.000000,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,None,None,None,0,2,2009,WD,Normal,3416 WEST ST AMES IA 50014,"HUBERT, JOHN F & GRETTA R","(3416, West Street, Union Drive, Ames, Story C...","(42.024855315930424, -93.66367099341755)",42.024855,-93.663671
2,1049,139500,120,RL,42.000000,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,None,None,None,0,2,2009,WD,Normal,3416 WEST ST AMES IA 50014,"HUBERT, JOHN F & GRETTA R","(3416, West Street, Union Drive, Ames, Story C...","(42.024855315930424, -93.66367099341755)",42.024855,-93.663671
3,1001,124900,30,C (all),60.000000,6060,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,None,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,None,None,None,0,11,2007,WD,Normal,320 S 2ND ST AMES IA 50010,"SKLENAR, TIM","(320, South 2nd Street, Ames, Story County, Io...","(42.021547615297024, -93.61406809992513)",42.021548,-93.614068
4,1039,114000,70,RL,80.000000,8146,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,None,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,None,None,None,0,5,2009,WD,Normal,1524 DOUGLAS AVE AMES IA 50010,"MILLARD, TAMMY","(1524, Douglas Avenue, Broadmoor, Ames, Story ...","(42.03739146938776, -93.61220695918368)",42.037391,-93.612207


Stack train and test set to be able to apply all transformations to both datasets. In addition, the test set has missing data in more features, thus these need to be handled.

In [185]:
#Filling in missing values (see work in the apdx_miss_feat.ipynb for exploration of  missing features)
#This cell is here for easy reload
def impute_missing(alldata):
    #Fill in the missing lats and logs with their mean
    alldata['lats']=np.where(alldata.lats.isna(), mean_lats, alldata.lats)
    alldata['longs']=np.where(alldata.longs.isna(), mean_longs, alldata.longs)

    

    
    alldata['PoolQC']=np.where(alldata.PoolQC.isna(), None, alldata.PoolQC)

    alldata['MiscFeature']=np.where(alldata.MiscFeature.isna(), None, alldata.MiscFeature)


    alldata['Electrical']=np.where(alldata.Electrical.isna(), 'SBrkr', alldata.Electrical)

    alldata['MasVnrType']=np.where(alldata.MasVnrType.isna(), None, alldata.MasVnrType)

    alldata['MasVnrArea']=np.where(alldata.MasVnrArea.isna(), 0, alldata.MasVnrArea)




    alldata['Alley']=np.where(alldata.Alley.isna(), None, alldata.Alley)

    alldata['Fence']=np.where(alldata.Fence.isna(), None, alldata.Fence)

    alldata['FireplaceQu']=np.where(alldata.FireplaceQu.isna(), None, alldata.FireplaceQu) #FireplaceQu:fireplace quality



    #Replace year garage was built with year house was built. The feature of not having a garage will need to be picked up by categorical garage variables
    #Can consider replacing it with 0 to increase the effect of having no garage, but may produce outliers this way
    alldata['GarageYrBlt']=np.where(alldata.GarageYrBlt.isna(), alldata.YearBuilt, alldata.GarageYrBlt) #FireplaceQu:fireplace quality
    #KNN imputer? Scikit learn base imputers?->Simple and KNN imputers
    #https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute

    for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
        alldata[col]=np.where(alldata[col].isna(), None, alldata[col]) 


    for col in ['GarageArea', 'GarageCars']:
        alldata[col]=np.where(alldata[col].isna(), 0, alldata[col]) 

    for col in ['BsmtFinType2', 'BsmtExposure', 'BsmtQual', 'BsmtCond','BsmtFinType1']:
        alldata[col]=np.where(alldata[col].isna(), None, alldata[col]) 

    for col in ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']:
        alldata[col]=np.where(alldata[col].isna(), 0, alldata[col])    

    def fillna_linear_reg(lin_reg, frontage, area):
        area_ = area.loc(frontage.isna())
        pred_frontage = lin_reg.predict(area)
        frontage.loc[nan_mask] = pred_frontage


    #The transformation below is only applied to the training set
    from sklearn.linear_model import LinearRegression    
    def fillna_linear_reg(X,y,data,mask=None):
        """Fill in the values in one dependent variable based on linear regression off another one"""
        df_=data[mask]
        df_na= data[~mask]
        lin_reg=LinearRegression().fit(df_[[X]],df_[[y]])
        preds=lin_reg.predict(df_na[[X]])
        data.loc[~mask,y]=np.squeeze(preds)

    fillna_linear_reg('LotArea','LotFrontage',alldata,mask=alldata['LotFrontage'].notnull())

    #Drop the id column as it is unique for each data point and hence is not useful for prediction
    if KAGGLE: 
        alldata.drop(['Id'], axis=1, inplace=True)
    else:
        alldata.drop(['PID'],axis=1, inplace=True)


    #Some columns only have missing values in the test set.  These are treated below.

    #This feature imputation is inspired by https://www.kaggle.com/code/lavanyashukla01/how-i-made-top-0-3-on-a-kaggle-competition
    alldata['MSZoning'] = alldata.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))

    alldata['Utilities']=np.where(alldata.Utilities.isna(), 'AllPub', alldata.Utilities)

    alldata['Functional']=np.where(alldata.Functional.isna(), 'Typ', alldata.Functional)

    alldata['Exterior1st']=np.where(alldata.Exterior1st.isna(), alldata['Exterior1st'].mode()[0], alldata.Exterior1st)
    alldata['Exterior2nd']=np.where(alldata.Exterior2nd.isna(), alldata['Exterior2nd'].mode()[0], alldata.Exterior2nd)

    for col in ['BsmtFullBath', 'BsmtHalfBath']:
        alldata[col]=np.where(alldata[col].isna(), 0, alldata[col])  

    alldata['KitchenQual']=np.where(alldata.KitchenQual.isna(), alldata['KitchenQual'].mode()[0], alldata.KitchenQual)
    alldata['SaleType']=np.where(alldata.SaleType.isna(), alldata['SaleType'].mode()[0], alldata.SaleType)
impute_missing(alldata2)

In [203]:
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0

lat1 = radians(42.018564)
lon1 = radians(-93.651619)
lat2 = radians(42.024855)
lon2 = radians(-93.663671)

dlon = lon2 - lon1
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
#print("Should be:", 278.546, "km")

Result: 1.217134515008795


In [198]:
alldata2.head(50)

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,lats,longs,MissingLoc
0,856,126000,30,RL,65.556308,7890,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,None,None,None,0,3,2010,WD,Normal,42.018564,-93.651619,0
1,1049,139500,120,RL,42.000000,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,None,None,None,0,2,2009,WD,Normal,42.024855,-93.663671,0
2,1049,139500,120,RL,42.000000,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,None,None,None,0,2,2009,WD,Normal,42.024855,-93.663671,0
3,1001,124900,30,C (all),60.000000,6060,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,None,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,None,None,None,0,11,2007,WD,Normal,42.021548,-93.614068,0
4,1039,114000,70,RL,80.000000,8146,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,None,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,None,None,None,0,5,2009,WD,Normal,42.037391,-93.612207,0
5,1665,227000,60,RL,70.000000,8400,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,None,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,None,None,None,0,11,2009,WD,Normal,42.044554,-93.631818,0
6,1922,198500,85,RL,64.000000,7301,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,Edwards,Norm,Norm,1Fam,SFoyer,7,5,2003,2003,Gable,CompShg,HdBoard,HdBoard,BrkFace,500.0,Gd,TA,Slab,None,None,None,None,0.0,None,0.0,0.0,0.0,GasA,Ex,Y,SBrkr,495,1427,0,0.0,0.0,3,0,4,1,Gd,7,Typ,1,Ex,BuiltIn,2003.0,RFn,2.0,672.0,TA,TA,Y,0,0,177,0,0,0,None,None,None,0,7,2009,ConLD,Normal,42.018988,-93.671572,0
7,936,93000,20,RM,60.000000,6000,Pave,Pave,Reg,Bnk,AllPub,Inside,Mod,OldTown,Norm,Norm,2fmCon,1Story,4,4,1953,1953,Gable,CompShg,MetalSd,MetalSd,None,0.0,Fa,TA,CBlock,Fa,TA,No,Unf,0.0,Unf,0.0,936.0,936.0,GasA,TA,N,SBrkr,936,0,0,0.0,0.0,1,0,2,1,TA,4,Min2,0,None,Detchd,1974.0,Unf,2.0,576.0,TA,TA,Y,0,32,112,0,0,0,None,None,None,0,2,2009,WD,Normal,42.029368,-93.651464,1
8,1246,187687,20,RL,53.000000,3710,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,1Fam,1Story,7,5,2007,2008,Gable,CompShg,WdShing

In [199]:
alldata2.to_csv("./data/Ames_HousePriceLocs.csv",index=False)

If working with the Kaggle version of the dataset, can use the function below to do train-test split